<a href="https://colab.research.google.com/github/raghulchandramouli/Data-Generator/blob/main/COCO_Modeler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies

In [ ]:
import os
import requests
import zipfile
import cv2
import numpy as np
import random
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline
import torch
from tqdm import tqdm

# Configuration

In [ ]:
DATASET_DIR = "dataset" # Directory where the COCO Dataset and generated files will be stored
COCO_URL = "http://images.cocodataset.org/zips/val2017.zip" # URL to download
COCO_ZIP = os.path.join(DATASET_DIR, "coco_val2017.zip") # Path of ZIP
COCO_DIR = os.path.join(DATASET_DIR, "val2017") # Path where COCO will be extracted
MASK_DIR = os.path.join(DATASET_DIR, "masks") # Directory to store the Generated masks.
INPAINTED_DIR = os.path.join(DATASET_DIR, "inpainted") # Directory to store the inpainted images
NUM_IMAGES_TO_PROCESS = 5000

In [ ]:
INPAINTED_ZIP = "/content/dataset/inpainted.zip"
MASK_ZIP = "/content/dataset/masks.zip"

In [ ]:
# Setting up DIRS:
os.makedirs(DATASET_DIR, exist_ok=True)

os.makedirs(MASK_DIR, exist_ok=True)
os.makedirs(INPAINTED_DIR, exist_ok=True)

In [ ]:
# --- Download and Extract COCO Dataset ---
if not os.path.exists(COCO_ZIP):  # Check if the COCO ZIP file has already been downloaded.
    print("Downloading COCO dataset...")
    response = requests.get(COCO_URL, stream=True)  # Download the COCO ZIP file from the URL.  stream=True allows for efficient downloading of large files.
    with open(COCO_ZIP, "wb") as file:  # Open the file in write-binary mode ('wb') to save the downloaded data.
        for chunk in response.iter_content(chunk_size=1024):  # Iterate over the data in chunks of 1024 bytes.
            file.write(chunk)  # Write each chunk to the file.

if not os.path.exists(COCO_DIR):
  print("Extracting COCO Dataset")
  with zipfile.ZipFile(COCO_ZIP, "r") as zip_ref:
    zip_ref.extractall(DATASET_DIR)

Extracting COCO Dataset


In [ ]:
# Unzipping Inpainted and Masks files:

if os.path.exists(INPAINTED_ZIP):
  print("Extracting Inpainted Images")
  with zipfile.ZipFile(INPAINTED_ZIP, "r") as zip_ref:
    for member in zip_ref.infolist():
        # Extract to the target path, ensuring the directory structure is preserved
        zip_ref.extract(member, os.path.join(DATASET_DIR, "inpainted"))

In [ ]:
if os.path.exists(MASK_ZIP):
  print("Extracting Masks")
  with zipfile.ZipFile(MASK_ZIP, "r") as zip_ref:
    for member in zip_ref.infolist():
      zip_ref.extract(member, os.path.join(DATASET_DIR, "masks"))

# Masks Generation - Helper Functions

In [ ]:
def generate_mask(image_path, mask_path):
  """
  Generates a binary masks with a randomly selected region for inpainting

  Args:
    image_path (str): Path to the input image.
    mask_path (str): Path to save the generated mask.
  """

  image = cv2.imread(image_path)
  h, w, _ = image.shape # `_` is used to disregard the num of channels since we are dealing with Binary.

  mask = np.zeros((h, w), dtype=np.uint8) # Initialize a black, uint8 signifies an a 8-bit integers from 0-255

  # Ensure x & y are within the image boundaries and allows masks size:
  x = random.randint(50, max(50, w - 100 - 50)) # generates a random x-coordinate for the top-left corner of the mask.

  y = random.randint(50, max(50, h - 100 - 50)) # generates a random y-coordinate for the top-left corner of the mask.

  # generate a random size for the Sq.mask, between 50 & 150.
  mask_size = random.randint(50, min(150, w - x - 50, h - y - 50))

  # Draw a white (255) rectangle on the mask, starting at (x, y) and with a size of mask_size x mask_size.  The -1 fills the rectangle.
  cv2.rectangle(mask, (x, y), (x + mask_size, y + mask_size), 255, -1)

  kernel = np.ones((15, 15), dtype=np.uint8) # Create a 15 x 15 kernel of ones, used for dilation
  mask = cv2.dilate(mask, kernel, iterations=5)  # Dilate the mask.  Dilation expands the white area of the mask, making the masked region larger.

  cv2.imwrite(mask_path, mask)  # Save the generated mask to the specified path as a PNG file.

# Inpainting Images - Helper Functions

In [ ]:
def inpaint_image_batch(original_paths, mask_paths, output_paths):
    """
    Batch inpainting using Stable Diffusion.

    Args:
        original_paths (List[str]): List of paths to original images.
        mask_paths (List[str]): List of paths to corresponding mask images.
        output_paths (List[str]): List of paths to save inpainted images.
    """
    originals = [Image.open(p).convert("RGB") for p in original_paths]
    masks = [Image.open(p).convert("L") for p in mask_paths]

    prompt = "Fill the missing area with a natural and most appealing background"
    batch_size = len(originals)
    prompts = [prompt] * batch_size  # same prompt for each image

    result = pipe(prompt=prompts, image=originals, mask_image=masks).images

    for img, out_path in zip(result, output_paths):
        img.save(out_path)


# Main-Script

In the `main_script function`, the code iterates through each image in the COCO dataset (specifically, the first 2000 images, thanks to coco_images = `coco_images[:NUM_IMAGES_TO_PROCESS]`).  Inside this loop, for each image:

1. **generate_mask(img_path, mask_path) is called:**

>The `img_path` (path to the original COCO image) and `mask_path` (path where the mask will be saved) are passed to the generate_mask function.

>This function creates a mask for the current image and saves it to the `MASK_DIR.`

2. **`inpaint_image(img_path, mask_path, inpainted_path)` is called:**

>Immediately after the mask is generated and saved, the code calls the `inpaint_image` function.

>This function takes the  `img_path`  (path to the original COCO image), the  `mask_path` (path to the mask just generated), and  `inpainted_path`  (path where the inpainted image will be saved) as arguments.

It then uses the Stable Diffusion pipeline (`pipe`) to inpaint the original image, using the generated mask, and saves the result to `INPAINTED_DIR`.

**In summary:** For each image, the mask is generated first, and then that mask is used to inpaint the image.  This ensures that there is a one-to-one correspondence between the original images, the masks, and the inpainted images, which is crucial for your SegFormer model.

**Getting the model from Stability**

In [ ]:
# load stable diffusion inpainting model:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
)
pipe = pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model_index.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
INPAINTED_DIR = "/content/dataset/inpainted"  # Corrected path
MASK_DIR = "/content/dataset/masks"  # Corrected path

In [ ]:
coco_images = sorted(os.listdir(COCO_DIR)) # gets a sorted list of all files
coco_images = coco_images[2588 : 2588 + (NUM_IMAGES_TO_PROCESS - 2411)] # takes the first NUM_IMAGES_TO_PROCESS `2000 to be precise`

In [ ]:
BATCH_SIZE = 8

for i in tqdm(range(0, len(coco_images), BATCH_SIZE), desc="Processing Batches"):
    batch_imgs = coco_images[i : i + BATCH_SIZE]

    img_paths = [os.path.join(COCO_DIR, img) for img in batch_imgs]
    mask_paths = [os.path.join(MASK_DIR, img.replace(".jpg", "_mask.png")) for img in batch_imgs]
    inpainted_paths = [os.path.join(INPAINTED_DIR, img.replace(".jpg", "_inpainted.jpg")) for img in batch_imgs]

    # Generate masks first:
    for img_path, mask_path in zip(img_paths, mask_paths):
        generate_mask(img_path, mask_path)

    # Inpaint images:
    inpaint_image_batch(img_paths, mask_paths, inpainted_paths)

Processing Batches:   0%|          | 0/302 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches:   0%|          | 1/302 [00:52<4:22:02, 52.23s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches:   1%|          | 2/302 [01:46<4:27:57, 53.59s/it]

...

Processing Batches:  99%|█████████▉| 300/302 [4:29:18<01:47, 53.88s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches: 100%|█████████▉| 301/302 [4:30:12<00:53, 53.86s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches: 100%|██████████| 302/302 [4:30:40<00:00, 53.78s/it]


In [ ]:
len(os.listdir(INPAINTED_DIR))

2412

# Converting it to ZIP format

In [ ]:
import shutil
import os

In [ ]:
shutil.make_archive("inpainted_2", 'zip', INPAINTED_DIR)
shutil.make_archive("masks_2", "zip", MASK_DIR )

'/content/masks_2.zip'

# Putting ZIP files togather

**For Inpainted Images**

In [ ]:
import zipfile
import shutil
import os

# Define the paths to your zip files and the output zip file
zip_file1 = "/content/inpainted.zip"
zip_file2 = "/content/inpainted_2.zip"
output_zip_file = "/content/combined_inpainted.zip"

# Create a temporary directory to extract the files
temp_extraction_inpainted = "/content/temp_extraction_inpainted"
os.makedirs(temp_extraction_inpainted, exist_ok=True)

# Function to extract zip file contents to a directory
def extract_zip(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

# Extract contents of both zip files to the temporary directory
extract_zip(zip_file1, temp_extraction_inpainted)
extract_zip(zip_file2, temp_extraction_inpainted)

# Create the output zip file, adding all files from the temporary directory
shutil.make_archive(output_zip_file.replace(".zip", ""), 'zip', temp_extraction_inpainted)

# Remove the temporary directory (optional)
#shutil.rmtree(temp_dir)

print(f"Combined zip file created at: {output_zip_file}")

Combined zip file created at: /content/combined_inpainted.zip


In [ ]:
len(os.listdir(temp_extraction_inpainted))

4411

**For Masks**

In [ ]:
import zipfile
import shutil
import os

# Define the paths to your zip files and the output zip file for MASKS
zip_file1_masks = "/content/masks.zip"
zip_file2_masks = "/content/masks_2.zip"
output_zip_file_masks = "/content/combined_masks.zip"

# Create a temporary directory to extract the mask files
temp_extraction_masks = "/content/temp_extraction_masks"  # Assigning the value
os.makedirs(temp_extraction_masks, exist_ok=True)

# Function to extract zip file contents to a directory (same as before)
def extract_zip(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

# Extract contents of both mask zip files to the temporary directory
extract_zip(zip_file1_masks, temp_extraction_masks)
extract_zip(zip_file2_masks, temp_extraction_masks)

# Create the output zip file for masks, adding all files from the temporary directory
shutil.make_archive(output_zip_file_masks.replace(".zip", ""), 'zip', temp_extraction_masks)

print(f"Combined masks zip file created at: {output_zip_file_masks}")

Combined masks zip file created at: /content/combined_masks.zip


In [ ]:
# You can now list the contents of the temp_extraction_masks directory:
len(os.listdir(temp_extraction_masks))

4411